# PLIP — Protein ligand interaction profiling

This notebook will walk us through using PLIP for identification of non-covalent interactions between biological macromolecules and their ligands.

**PLIP implementation paper**:
Adasme et al. PLIP 2021: expanding the scope of the protein-ligand interaction profiler to DNA and RNA. NAR 2021

# 0) Complete example
See the [sample notebook](/Quickstarts/protein-ligand-interactions-sample.ipynb) for a quick demonstration.

# 1) Setup

## 1.0) Imports

In [ ]:
import os
import json
from pathlib import Path

import rush
from pdbtools import pdb_fetch

## 1.1) Configuration

Let's set some global variables that define our project.

In [ ]:
DESCRIPTION = "plip-inference-notebook"
TAGS = ["rush-py", "plip", "notebook"]

## 1.2) Build your client

In [ ]:
# |hide
WORK_DIR = Path.home() / "qdx" / "plip-inference-rush-py-demo"
if WORK_DIR.exists():
    client = rush.Provider(workspace=WORK_DIR)
    await client.nuke(remote=False)
os.makedirs(WORK_DIR, exist_ok=True)
YOUR_TOKEN = os.getenv("RUSH_TOKEN")

2024-05-07 21:33:29,577 - rush - WARNING - Module github:talo/tengu-plip/790c01804c0942fb11df8bc91c5d3d983ca092af#plip_tengu_pdb is not in the lock file
2024-05-07 21:33:29,577 - rush - WARNING - Module plip has a different version on the server: github:talo/tengu-plip/790c01804c0942fb11df8bc91c5d3d983ca092af#plip_tengu.
                                Use `.update_modules()` to update the lock file


In [ ]:
os.environ["RUSH_TOKEN"] = YOUR_TOKEN
client = rush.build_blocking_provider_with_functions(
    batch_tags=TAGS, workspace=WORK_DIR
)

# 2) Preparation

## 2.0) Fetch example PDB
Note that PLIP requires a PDB file of a protein-ligand complex.

In [ ]:
PDB_ID = "1GIH"
FILE_NAME = f"{PDB_ID}.pdb"
FILE_PATH = WORK_DIR / FILE_NAME

In [ ]:
complex = list(pdb_fetch.fetch_structure(PDB_ID))

with open(FILE_PATH, "w") as f:
    for line in complex:
        f.write(line)

# 3) PLIP

## 3.1) Arguments

In [ ]:
help(client.plip_pdb)

Help on function plip_pdb in module rush.provider:

plip_pdb(*args: *tuple[RushObject[bytes]], target: 'Target | None' = None, resources: 'Resources | None' = None, tags: 'list[str] | None' = None, restore: 'bool | None' = None) -> tuple[RushObject[Record], RushObject[bytes]]
    Run PLIP on the input PDB file of a protein-ligand complex to obtain interaction data.


    Module version:
    `github:talo/tengu-plip/790c01804c0942fb11df8bc91c5d3d983ca092af#plip_tengu_pdb`

    QDX Type Description:

        input_complex_pdb_file: Object {
            size: u32,
            path: @$PDB,
            format: ObjectFormat[json | bin]?
        }
        ->
        output_conformer: Object[Conformer];
        output_txt_file: Object[@$PDB]


    :param input_complex_pdb_file: The protein-ligand complex as a PDB file; ligand should be the only HETATM lines.
    :return output_conformer: A QDX Conformer of the complex with the computed interaction data
    :return output_txt_file: The output tx

We can see from the above help documentation that we need to pass the Path to our PDB file as an argument.

## 3.2) Run PLIP
Finally, we run PLIP so we can identify interactions between the ligand and the protein within our protein complex.

In [ ]:
PLIP_RESOURCES = {"storage": 1024_000}

output_conformer, output_txt = client.plip_pdb(FILE_PATH, resources=PLIP_RESOURCES)

## 3.3) Get output conformer
Here, we get the output Conformer (in QDXF format) that is enriched with `binding_site_interactions` by PLIP.

In [ ]:
output_conformer = json.load(output_conformer.download().open())

2024-05-07 21:33:35,990 - rush - INFO - Argument c3bed94c-2586-46fb-97db-eeab1217c00c is now ModuleInstanceStatus.RESOLVING
2024-05-07 21:33:40,599 - rush - INFO - Argument c3bed94c-2586-46fb-97db-eeab1217c00c is now ModuleInstanceStatus.ADMITTED
2024-05-07 21:33:46,334 - rush - INFO - Argument c3bed94c-2586-46fb-97db-eeab1217c00c is now ModuleInstanceStatus.DISPATCHED
2024-05-07 21:33:47,497 - rush - INFO - Argument c3bed94c-2586-46fb-97db-eeab1217c00c is now ModuleInstanceStatus.AWAITING_UPLOAD


In [ ]:
output_conformer["binding_site_interactions"]

[{'kind': 'Hydrophobic',
  'ligand_residue_idx': 501,
  'ligand_atom_idx': 2227,
  'ligand_xyz': [5.034, 10.623, 29.029],
  'receptor_amino_acid_idx': 18,
  'receptor_atom_idx': 140,
  'receptor_xyz': [6.814, 7.439, 30.53],
  'receptor_is_donor': False,
  'receptor_is_charged': False,
  'receptor_is_positively_charged': False,
  'pi_stack_kind': None},
 {'kind': 'Hydrophobic',
  'ligand_residue_idx': 501,
  'ligand_atom_idx': 2233,
  'ligand_xyz': [3.287, 9.093, 27.493],
  'receptor_amino_acid_idx': 31,
  'receptor_atom_idx': 242,
  'receptor_xyz': [3.539, 5.677, 28.924],
  'receptor_is_donor': False,
  'receptor_is_charged': False,
  'receptor_is_positively_charged': False,
  'pi_stack_kind': None},
 {'kind': 'Hydrophobic',
  'ligand_residue_idx': 501,
  'ligand_atom_idx': 2238,
  'ligand_xyz': [2.847, 9.698, 28.682],
  'receptor_amino_acid_idx': 64,
  'receptor_atom_idx': 443,
  'receptor_xyz': [-0.699, 10.68, 27.296],
  'receptor_is_donor': False,
  'receptor_is_charged': False,
  '

## 3.4) Get PLIP output document
PLIP also returns a link to the output document of the results of the PLIP output.

In [ ]:
output_txt.download("plip_output.txt", overwrite=True).open().readline()

'Prediction of noncovalent interactions for PDB structure 1GIH\n'